# Tarea 1 - Taller de Deep Learning

**Fecha de entrega: 19/10/2025**  

In [1]:
import torchvision.datasets as datasets
from pathlib import Path
import torch
import torch.nn as nn
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader, random_split
import utils
import sys
import torch.optim as optim


la ki
cb879b8473582418585ac8a9fcb1e1695ebcdf97

In [2]:
import wandb

In [3]:
WANDB_TEAM_NAME = "[joacoguerrae-ort]"
WANDB_PROJECT = "[wandb-labo1-guerra-sanes]"
wandb.login()

wandb: Currently logged in as: joacoguerrae (joacoguerrae-ort) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
DEVICE = "cpu"  # por defecto, usamos la CPU
if torch.cuda.is_available():
    DEVICE = "cuda"  # si hay GPU, usamos la GPU
elif torch.backends.mps.is_available():
    DEVICE = "mps"  # si no hay GPU, pero hay MPS, usamos MPS
elif torch.xpu.is_available():
    DEVICE = "xpu"  # si no hay GPU, pero hay XPU, usamos XPU

print(f"Usando {DEVICE}")

NUM_WORKERS = 0 # Win y MacOS pueden tener problemas con múltiples workers
if sys.platform == 'linux':
    NUM_WORKERS = 4  # numero de workers para cargar los datos (depende de cada caso)

print(f"Usando {NUM_WORKERS}")

Usando cuda
Usando 4


In [5]:
DATA_DIR = Path("data")



imagenette_dataset = datasets.Imagenette(
    DATA_DIR, download=False
)

transforms = T.Compose([
    T.ToImage(),
    T.Grayscale(),
    T.ToDtype(torch.float32, scale=True),
    T.Resize((224, 224))

])

imagenette_dataset_train = datasets.Imagenette(
    DATA_DIR, download=False,split = 'train', transform=transforms
)

imagenette_dataset_val = datasets.Imagenette(
    DATA_DIR, download=False,split = 'val', transform=transforms
)

name_classes = imagenette_dataset.classes
nclasses = len(name_classes)


In [6]:
imagenette_dataset_train, imagenette_dataset_test = random_split(
    imagenette_dataset_train, [0.8, 0.2]
)

In [7]:
BATCH_SIZE = 128  # tamaño del batch
def get_dataloaders(batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
    train_loader = DataLoader(
        imagenette_dataset_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    val_loader = DataLoader(
        imagenette_dataset_val,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    test_loader = DataLoader(
        imagenette_dataset_test,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
    )

    return train_loader, val_loader, test_loader


train_loader, val_loader, test_loader = get_dataloaders()

In [8]:
class RegularizedCNN(nn.Module):
    def __init__(self, input, n_classes, dropout_rate=0.5):
        super().__init__()
        self.conv = nn.Sequential(
            # 224*224
            nn.Conv2d(input[0], 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Batch Normalization
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.1),  # Dropout espacial
            
            # 112*112
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),
            
            # 56*56
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.25),
            # 28*28
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            
            nn.Linear(256, n_classes),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x

# Crear el modelo con regularización
modelo_regularizado = RegularizedCNN(train_loader.dataset[0][0].shape, nclasses, dropout_rate=0.5).to(DEVICE)
print(modelo_regularizado)

# Optimizer con weight decay (L2 regularization)
optimizer_reg = optim.Adam(modelo_regularizado.parameters(), lr=1e-3, weight_decay=1e-4)

RegularizedCNN(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.1, inplace=False)
    (5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout2d(p=0.25, inplace=False)
  )
  

In [9]:
# Configuración mejorada del sweep con más parámetros
sweep_config_improved = {
    "name": "sweep-imagenette-regularized",
    "method": "bayes",  # Bayesian optimization es más eficiente que random
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {
            "distribution": "log_uniform_values",
            "min": 1e-5,
            "max": 1e-2
        },
        "optimizer": {
            "values": ["adam", "sgd", "adamw"]
        },
        "batch_size": {
            "values": [32, 64, 128, 256]
        },
        "dropout_rate": {
            "distribution": "uniform",
            "min": 0.2,
            "max": 0.7
        },
        "weight_decay": {
            "distribution": "log_uniform_values",
            "min": 1e-6,
            "max": 1e-3
        },
        "momentum": {
            "distribution": "uniform",
            "min": 0.85,
            "max": 0.99
        },
        "epochs": {
            "value": 30
        }
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 5
    }
}



In [10]:
def train_with_wandb(config=None):
    """
    Función de entrenamiento que será llamada por wandb.agent para cada configuración del sweep.
    """
    with wandb.init(config=config):
        # Obtener la configuración del sweep
        config = wandb.config
        
        # Crear dataloaders con el batch_size del sweep
        train_loader_sweep, val_loader_sweep, test_loader_sweep = get_dataloaders(
            batch_size=config.batch_size, 
            num_workers=NUM_WORKERS
        )
        
        # Crear el modelo con los parámetros del sweep
        model = RegularizedCNN(
            train_loader_sweep.dataset[0][0].shape, 
            nclasses, 
            dropout_rate=config.dropout_rate
        ).to(DEVICE)
        
        # Definir el optimizador según la configuración
        if config.optimizer == "adam":
            optimizer = optim.Adam(
                model.parameters(), 
                lr=config.learning_rate, 
                weight_decay=config.weight_decay
            )
        elif config.optimizer == "sgd":
            optimizer = optim.SGD(
                model.parameters(), 
                lr=config.learning_rate, 
                momentum=config.momentum,
                weight_decay=config.weight_decay
            )
        elif config.optimizer == "adamw":
            optimizer = optim.AdamW(
                model.parameters(), 
                lr=config.learning_rate, 
                weight_decay=config.weight_decay
            )
        
        # Criterio de pérdida
        criterion = nn.CrossEntropyLoss()
        
        # Entrenar el modelo
        train_errors, val_errors = utils.train(
            model, 
            optimizer=optimizer, 
            criterion=criterion, 
            do_early_stopping=True,
            train_loader=train_loader_sweep, 
            val_loader=val_loader_sweep, 
            device=DEVICE, 
            epochs=config.epochs,
            patience=10
        )
        
        # Evaluar en el conjunto de test al final
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader_sweep:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        test_accuracy = 100. * correct / total
        test_loss = test_loss / len(test_loader_sweep)
        
        # Log de métricas finales
        wandb.log({
            "test_accuracy": test_accuracy,
            "test_loss": test_loss
        })
        
        print(f"Test Accuracy: {test_accuracy:.2f}%")

In [11]:
# Crear el sweep
sweep_id_improved = wandb.sweep(sweep_config_improved, project=WANDB_PROJECT)

print(f"Sweep ID: {sweep_id_improved}")
print(f"Para ejecutar el sweep, usa: wandb.agent('{sweep_id_improved}', function=train_with_wandb, count=20)")

Create sweep with ID: 9zt0jvn4
Sweep URL: https://wandb.ai/joacoguerrae-ort/%5Bwandb-labo1-guerra-sanes%5D/sweeps/9zt0jvn4
Sweep ID: 9zt0jvn4
Para ejecutar el sweep, usa: wandb.agent('9zt0jvn4', function=train_with_wandb, count=20)


In [12]:
wandb.agent(sweep_id_improved, function=train_with_wandb, count=20)

wandb: Agent Starting Run: ntjh95ra with config:
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.3644549441792566
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0002100757771524526
wandb: 	momentum: 0.9839020638084366
wandb: 	optimizer: adam
wandb: 	weight_decay: 9.53221745438804e-06


Epoch: 001 | Train Loss: 2.03762 | Val Loss: 2.70280
Epoch: 002 | Train Loss: 1.79087 | Val Loss: 1.93354
Epoch: 003 | Train Loss: 1.65645 | Val Loss: 1.62421
Epoch: 004 | Train Loss: 1.53419 | Val Loss: 1.50218
Epoch: 005 | Train Loss: 1.42616 | Val Loss: 1.42095
Epoch: 006 | Train Loss: 1.32148 | Val Loss: 1.37117
Epoch: 007 | Train Loss: 1.20626 | Val Loss: 1.34193
Epoch: 008 | Train Loss: 1.10426 | Val Loss: 1.31068
Epoch: 009 | Train Loss: 0.99851 | Val Loss: 1.34576
Epoch: 010 | Train Loss: 0.88667 | Val Loss: 1.34744
Epoch: 011 | Train Loss: 0.77967 | Val Loss: 1.36706
Epoch: 012 | Train Loss: 0.66709 | Val Loss: 1.40856
Epoch: 013 | Train Loss: 0.55424 | Val Loss: 1.42268
Epoch: 014 | Train Loss: 0.47423 | Val Loss: 1.36721
Epoch: 015 | Train Loss: 0.39840 | Val Loss: 1.38039
Epoch: 016 | Train Loss: 0.32725 | Val Loss: 1.48383
Epoch: 017 | Train Loss: 0.27575 | Val Loss: 1.48539
Epoch: 018 | Train Loss: 0.23370 | Val Loss: 1.49680
Detener entrenamiento en la época 17, la mejor

test_accuracy,▁
test_loss,▁
test_accuracy,55.25621
test_loss,1.48622


wandb: Agent Starting Run: 28aagqmb with config:
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.2079046313573996
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0004587744668999234
wandb: 	momentum: 0.9453175681909808
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.00012401405791891623


Epoch: 001 | Train Loss: 2.17105 | Val Loss: 2.53195
Epoch: 002 | Train Loss: 1.87851 | Val Loss: 1.91974
Epoch: 003 | Train Loss: 1.74893 | Val Loss: 1.73606
Epoch: 004 | Train Loss: 1.66779 | Val Loss: 1.65808
Epoch: 005 | Train Loss: 1.58315 | Val Loss: 1.61134
Epoch: 006 | Train Loss: 1.51127 | Val Loss: 1.57495
Epoch: 007 | Train Loss: 1.44697 | Val Loss: 1.52384
Epoch: 008 | Train Loss: 1.39197 | Val Loss: 1.49613
Epoch: 009 | Train Loss: 1.33404 | Val Loss: 1.44872
Epoch: 010 | Train Loss: 1.27271 | Val Loss: 1.43125
Epoch: 011 | Train Loss: 1.21823 | Val Loss: 1.41799
Epoch: 012 | Train Loss: 1.15972 | Val Loss: 1.37082
Epoch: 013 | Train Loss: 1.10597 | Val Loss: 1.38001
Epoch: 014 | Train Loss: 1.05104 | Val Loss: 1.35527
Epoch: 015 | Train Loss: 1.00260 | Val Loss: 1.36608
Epoch: 016 | Train Loss: 0.95012 | Val Loss: 1.35605
Epoch: 017 | Train Loss: 0.89565 | Val Loss: 1.32097
Epoch: 018 | Train Loss: 0.84064 | Val Loss: 1.35385
Epoch: 019 | Train Loss: 0.79242 | Val Loss: 1

...

Epoch: 001 | Train Loss: 2.06207 | Val Loss: 2.73667
Epoch: 002 | Train Loss: 1.83366 | Val Loss: 1.94991
Epoch: 003 | Train Loss: 1.73753 | Val Loss: 1.75352
Epoch: 004 | Train Loss: 1.67686 | Val Loss: 1.68448
Epoch: 005 | Train Loss: 1.60643 | Val Loss: 1.66729
Epoch: 006 | Train Loss: 1.55693 | Val Loss: 1.63009
Epoch: 007 | Train Loss: 1.51003 | Val Loss: 1.61486
Epoch: 008 | Train Loss: 1.45849 | Val Loss: 1.58559
Epoch: 009 | Train Loss: 1.41263 | Val Loss: 1.57555
Epoch: 010 | Train Loss: 1.36373 | Val Loss: 1.55530
Epoch: 011 | Train Loss: 1.32648 | Val Loss: 1.52748
Epoch: 012 | Train Loss: 1.27717 | Val Loss: 1.52306
Epoch: 013 | Train Loss: 1.24603 | Val Loss: 1.50200
Epoch: 014 | Train Loss: 1.19800 | Val Loss: 1.48556
Epoch: 015 | Train Loss: 1.15467 | Val Loss: 1.48895
Epoch: 016 | Train Loss: 1.10561 | Val Loss: 1.49038
Epoch: 017 | Train Loss: 1.07294 | Val Loss: 1.47902
Epoch: 018 | Train Loss: 1.02721 | Val Loss: 1.45763
Epoch: 019 | Train Loss: 0.98434 | Val Loss: 1

test_accuracy,▁
test_loss,▁
test_accuracy,53.77707
test_loss,1.45357


wandb: Agent Starting Run: 3iqgnnez with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.689860424087003
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0008445394606376422
wandb: 	momentum: 0.9710794321333164
wandb: 	optimizer: adam
wandb: 	weight_decay: 2.1427356071176813e-05


Epoch: 001 | Train Loss: 2.18552 | Val Loss: 1.82543
Epoch: 002 | Train Loss: 1.89972 | Val Loss: 1.63312
Epoch: 003 | Train Loss: 1.74003 | Val Loss: 1.49574
Epoch: 004 | Train Loss: 1.61282 | Val Loss: 1.42595
Epoch: 005 | Train Loss: 1.54802 | Val Loss: 1.33412
Epoch: 006 | Train Loss: 1.45992 | Val Loss: 1.27880
Epoch: 007 | Train Loss: 1.37920 | Val Loss: 1.25754
Epoch: 008 | Train Loss: 1.32099 | Val Loss: 1.21243
Epoch: 009 | Train Loss: 1.24844 | Val Loss: 1.19854
Epoch: 010 | Train Loss: 1.19253 | Val Loss: 1.16928
Epoch: 011 | Train Loss: 1.13306 | Val Loss: 1.15311
Epoch: 012 | Train Loss: 1.08645 | Val Loss: 1.14978
Epoch: 013 | Train Loss: 1.01991 | Val Loss: 1.15834
Epoch: 014 | Train Loss: 0.96926 | Val Loss: 1.17678
Epoch: 015 | Train Loss: 0.89012 | Val Loss: 1.19001
Epoch: 016 | Train Loss: 0.83735 | Val Loss: 1.17874
Epoch: 017 | Train Loss: 0.76576 | Val Loss: 1.19832
Epoch: 018 | Train Loss: 0.69701 | Val Loss: 1.22169
Epoch: 019 | Train Loss: 0.67267 | Val Loss: 1

test_accuracy,▁
test_loss,▁
test_accuracy,60.85578
test_loss,1.30012


In [13]:
# Check sweep status and results
wandb.finish()  # Finish current run if any

# View the best run from your sweep
api = wandb.Api()
sweep = api.sweep(f"{WANDB_TEAM_NAME}/{WANDB_PROJECT}/{sweep_id_improved}")
best_run = sweep.best_run()

print(f"Best run: {best_run.name}")
print(f"Best validation loss: {best_run.summary['val_loss']}")
print(f"Best config: {best_run.config}")

# Train final model with best hyperparameters
# You can now use the best config to train your final model


CommError: Could not find sweep <Sweep %5Bjoacoguerrae-ort%5D/%5Bwandb-labo1-guerra-sanes%5D/9zt0jvn4 (Unknown State)>